# Join Similarity Calculation of MLB Tables

In [5]:
from pyspark import SparkConf
from pyspark.ml.stat import Summarizer
from pyspark.sql import DataFrameReader, DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType, IntegerType, StringType, DoubleType
from pyspark.sql.functions import col, isnan, when, count
from pyspark.mllib.linalg.distributed import RowMatrix
from helper_functions import print_df_to_html, translate_header_file_to_list, variations, pair_permutations_ordered, translate_datatype_file_to_list
from scipy.stats import wasserstein_distance
from numpy import asarray

In [7]:
translate_datatype_file_to_list("data/benchmark/MLB/samples/MLB_1.datatypes.csv")

[IntegerType,
 DecimalType(4,3),
 DecimalType(4,3),
 IntegerType,
 IntegerType,
 IntegerType,
 DecimalType(4,1),
 IntegerType,
 DecimalType(4,1),
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 DecimalType(4,3),
 IntegerType,
 DecimalType(4,1),
 IntegerType,
 IntegerType,
 DecimalType(4,3),
 IntegerType,
 DecimalType(4,1),
 IntegerType,
 IntegerType,
 IntegerType,
 DecimalType(4,3),
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 IntegerType,
 StringType,
 StringType,
 IntegerType,
 StringType,
 StringType,
 IntegerType,
 StringType,
 StringType,
 DecimalType(4,3),
 DecimalType(3,1),
 IntegerType,
 DecimalType(3,1),
 IntegerType,
 DecimalType(17,14),
 DecimalType(17,14)]

In [2]:
BENCHMARK_REL_PATH = "data/benchmark/"

#list_of_all_MLB_tables = [ "MLB_1", "MLB_20", "MLB_15"]
#list_of_all_MLB_tables = [ "MLB_1", "MLB_20"]
#list_of_all_MLB_tables = [ "MLB_1", "MLB_12", "MLB_13", "MLB_14" ]
list_of_all_MLB_tables = [
    "MLB_1", "MLB_10", "MLB_11", "MLB_12", "MLB_13", "MLB_14", "MLB_15",
    "MLB_16", "MLB_17", "MLB_18", "MLB_19", "MLB_2", "MLB_20", "MLB_21",
    "MLB_22", "MLB_23", "MLB_24", "MLB_25", "MLB_26", "MLB_27", "MLB_28",
    "MLB_29", "MLB_3", "MLB_30", "MLB_31", "MLB_32", "MLB_33", "MLB_34",
    "MLB_35", "MLB_36", "MLB_37", "MLB_38", "MLB_39", "MLB_4", "MLB_40",
    "MLB_41", "MLB_42", "MLB_43", "MLB_44", "MLB_45", "MLB_46", "MLB_47",
    "MLB_48", "MLB_49", "MLB_5", "MLB_50", "MLB_51", "MLB_52", "MLB_53",
    "MLB_54", "MLB_55", "MLB_56", "MLB_57", "MLB_58", "MLB_59", "MLB_6",
    "MLB_60", "MLB_61", "MLB_62", "MLB_63", "MLB_64", "MLB_65", "MLB_66",
    "MLB_67", "MLB_68", "MLB_7", "MLB_8", "MLB_9" ]
#list_of_MLB_join_candidate_pairs = [ ("MLB_1","MLB_12"), ("MLB_1","MLB_13"), ("MLB_1","MLB_14")  ]


## Setup the MLB Tables in Spark 

In [7]:
def cast_datatypes(datatype_file, input_df:DataFrame) -> DataFrame:
    datatype_list  = translate_datatype_file_to_list(datatype_file)
    df = input_df.alias('tmp_df')
    if len(df.columns) == len(datatype_list):
        for i in range(len(datatype_list)):
            if datatype_list[i] != type(df.schema[i].dataType) :
                #print (f"{df.columns[i]} {datatype_list[i]} {df.schema[i].dataType}") 
                df = df.withColumn(df.columns[i],col(f"`{df.columns[i]}`").cast(datatype_list[i]))                
        return df
    else:
        return df

In [8]:
def check_attribute_completeness(all_columns:list, string_attributes:list, numeric_attributes:list):
    for curr_col in all_columns:
        if curr_col not in string_attributes and curr_col not in numeric_attributes :
            print (f"{curr_col} is not numeric or string")

In [9]:
def compare_schemas(df1:DataFrame, df2:DataFrame):
    if len(df1.dtypes) != len(df2.dtypes):
        print ("Schemas have differen sizes!!!")
    else :
        for i in range(len(df1.dtypes)):
            if df1.dtypes[i][0] != df2.dtypes[i][0] or df1.dtypes[i][1] != df2.dtypes[i][1] :
                print (f"Columns differ {df1.dtypes[i]} {df2.dtypes[i]}")

In [10]:
file_path = BENCHMARK_REL_PATH+"MLB/"
sample = False
# create Spark Config
conf = SparkConf()
conf.set("spark.executor.memory", "4g")
conf.set("spark.driver.memory", "4g")
conf.set("spark.memory.offHeap.enabled","true" )
conf.set("spark.memory.offHeap.size","4g") 
conf.setMaster("local[2]")
conf.setAppName("MLB-similarity-calc")
# create a SparkSession
spark = SparkSession.builder.config(conf=conf).getOrCreate()
# dict of string attributes for each table
string_attributes = {}
numeric_attributes = {}
for table_name in list_of_all_MLB_tables:
       if sample : 
              data_file = file_path+"samples/"+table_name+".sample"+".csv"
       else :
              data_file = file_path+table_name+".csv"
       header_file = file_path+"samples/"+table_name+".header.csv"
       datatype_file = file_path+"samples/"+table_name+".datatypes.csv"
       # create a DataFrame using an ifered Schema 
       orig_df = spark.read.option("header", "false") \
       .option("inferSchema", "true") \
       .option("delimiter", "|") \
       .csv(data_file).toDF(*translate_header_file_to_list(header_file)) 
       df = cast_datatypes(datatype_file, orig_df)
       # compare_schemas(orig_df, df)
       df.createOrReplaceTempView(table_name)
       string_attributes[table_name] = list(filter(lambda x : not x.startswith("Calculation"), \
                                       map(lambda x : x[0], filter(lambda tupel: tupel[1] == 'string' ,df.dtypes))))
       numeric_attributes[table_name] = list(filter(lambda x : not x.startswith("Calculation"), \
                                          map(lambda x : x[0], \
                                              filter(lambda tupel: tupel[1] == 'double' or \
                                              tupel[1] == 'int' or tupel[1].startswith('decimal'),df.dtypes))))
       check_attribute_completeness(df.columns, string_attributes[table_name], numeric_attributes[table_name])
       


Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458117263387 is not numeric or string
Calculation_40532458113208334 is not numeric or string
Calculation_40532458117263387 is not numeric or string
Calculation_40532458113208334 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_40532458112880653 is not numeric or string
Calculation_40532458117070874 is not numeric or string
Calculation_496521908233621517 is not numeric or string
Calculation_40532458112270348 is not numeric or string
Calculation_40532458116673561 is not numeric or string
Calculation_496521908233621517 is not numeric or string
Calculation_40532458112270348 is not numeric or string
Calculation_40532458116673561 is not numeric or string
Calculat

## Wasserstein (earth mover) and Cos distance calculation

In [8]:
result_dist_calc = []
for curr_set in variations(list_of_all_MLB_tables,2):
    outer = curr_set.pop()
    inner = curr_set.pop()
    # find matching attributes to compare
    join_attributes = list(set(string_attributes[inner]) & set(string_attributes[outer])) 
    join_condition = "ON (" + " AND ".join(map(lambda join_att : f"o.`{join_att}` = i.`{join_att}`" ,\
                                           join_attributes))
    intersecting_attr = list(set(numeric_attributes[inner]) & set(numeric_attributes[outer])) 
    #create projection list
    projection_list = " , ".join(map(lambda attr : f"o.`{attr}` as `o.{attr}` , i.`{attr}` as `i.{attr}`", intersecting_attr))
    sqlDF = spark.sql("SELECT "+projection_list+" FROM " +outer +" o JOIN "+ \
                            inner+ " i " + join_condition+")")
    # filter out null tupels with null values
    sqlDF = sqlDF.dropna(subset=list(map(lambda cur_col : f"`{cur_col}`", sqlDF.columns)))
    # calculates null values in the table
    #sqlDF.select([count(when(isnan(f"`{c}`") | col(f"`{c}`").isNull(), c)).alias(f"`{c}`") for c in sqlDF.columns]).show()
    
    # calcultes basic statisitc for the attributes
    print_df_to_html(sqlDF.describe())

    attr_variations = pair_permutations_ordered(intersecting_attr)
    for curr_item in attr_variations :
        first_attr = curr_item[0]
        second_attr = curr_item[1]    
        ws_calc = wasserstein_distance( \
               asarray(sqlDF.select(col(f"`o.{first_attr}`")).rdd.flatMap(lambda x: x).collect()),                  asarray(sqlDF.select(col(f"`i.{second_attr}`")).rdd.flatMap(lambda x: x).collect()))
        first_second_projection = sqlDF.select(col(f"`o.{first_attr}`"),col(f"`i.{second_attr}`"))
        cos_calc = 0.0
        cos_sim = RowMatrix(first_second_projection.rdd.map(list)).columnSimilarities()
        if cos_sim.entries.count()> 0: 
            cos_calc = cos_sim.entries.first().value
        else:
            # print(f"Cos calc not possible {outer} {first_attr} {first_second_projection.dtypes[0]} {inner} {second_attr} {first_second_projection.dtypes[1]}")
            cos_calc = float("NaN")
        result_dist_calc.append([outer, first_attr, inner, second_attr, \
                                 float(ws_calc), float(cos_calc) ])
result_dist_calcdf = spark.createDataFrame(result_dist_calc).\
                           toDF("OUTER","OUTER_ATTR","INNER","INNER_ATTR","EMD", "COS")
result_dist_calcdf.coalesce(1).write.format("csv").mode("overwrite")\
                    .option("header","true").save("results/emd_cos_result_dist_calcs")
result_dist_calcdf.show()

KeyboardInterrupt: 

In [12]:
## Calc EMD and COS just for selected combinations

result_dist_calc = []
list_of_MLB_table_matches_to_calc__sim_metrics = [
    {'MLB_1', 'MLB_10'},
    {'MLB_1', 'MLB_11'},
    {'MLB_1', 'MLB_12'},
    {'MLB_1', 'MLB_13'},
    {'MLB_1', 'MLB_14'},
    {'MLB_1', 'MLB_15'},
    {'MLB_1', 'MLB_16'},
    {'MLB_1', 'MLB_17'},
    {'MLB_1', 'MLB_18'},
    {'MLB_1', 'MLB_19'},
    {'MLB_1', 'MLB_2'},
    {'MLB_1', 'MLB_20'},
    {'MLB_1', 'MLB_21'},
    {'MLB_1', 'MLB_22'},
    {'MLB_1', 'MLB_23'},
    {'MLB_1', 'MLB_24'},
    {'MLB_1', 'MLB_25'},
    {'MLB_1', 'MLB_26'},
    {'MLB_1', 'MLB_27'},
    {'MLB_1', 'MLB_28'},
    {'MLB_1', 'MLB_29'},
    {'MLB_1', 'MLB_3'},
    {'MLB_1', 'MLB_30'},
    {'MLB_1', 'MLB_31'},
    {'MLB_1', 'MLB_33'},
    {'MLB_1', 'MLB_34'},
    {'MLB_1', 'MLB_35'},
    {'MLB_1', 'MLB_36'},
    {'MLB_1', 'MLB_37'},
    {'MLB_1', 'MLB_38'},
    {'MLB_1', 'MLB_39'},
    {'MLB_1', 'MLB_4'},
    {'MLB_1', 'MLB_40'},
    {'MLB_1', 'MLB_41'},
    {'MLB_1', 'MLB_42'},
    {'MLB_1', 'MLB_43'},
    {'MLB_1', 'MLB_44'},
    {'MLB_1', 'MLB_45'},
    {'MLB_1', 'MLB_46'},
    {'MLB_1', 'MLB_47'},
    {'MLB_1', 'MLB_48'},
    {'MLB_1', 'MLB_49'},
    {'MLB_1', 'MLB_5'},
    {'MLB_1', 'MLB_50'},
    {'MLB_1', 'MLB_51'},
    {'MLB_1', 'MLB_52'},
    {'MLB_1', 'MLB_53'},
    {'MLB_1', 'MLB_54'},
    {'MLB_1', 'MLB_55'},
    {'MLB_1', 'MLB_56'},
    {'MLB_1', 'MLB_57'},
    {'MLB_1', 'MLB_58'},
    {'MLB_1', 'MLB_59'},
    {'MLB_1', 'MLB_6'},
    {'MLB_1', 'MLB_60'},
    {'MLB_1', 'MLB_61'},
    {'MLB_1', 'MLB_62'},
    {'MLB_1', 'MLB_63'},
    {'MLB_1', 'MLB_64'},
    {'MLB_1', 'MLB_65'},
    {'MLB_1', 'MLB_66'},
    {'MLB_1', 'MLB_67'},
    {'MLB_1', 'MLB_68'},
    {'MLB_1', 'MLB_7'},
    {'MLB_1', 'MLB_8'},
    {'MLB_1', 'MLB_9'}
]

for index, curr_set in enumerate(list_of_MLB_table_matches_to_calc__sim_metrics):
    # if index > 1:
    #     break
    outer = curr_set.pop()
    inner = curr_set.pop()
    #print(inner)
    #print(outer)
    #print(index)
    # find matching attributes to compare
    join_attributes = list(set(string_attributes[inner]) & set(string_attributes[outer])) 
    join_condition = "ON (" + " AND ".join(map(lambda join_att : f"o.`{join_att}` = i.`{join_att}`" ,\
                                           join_attributes))
    intersecting_attr = list(set(numeric_attributes[inner]) & set(numeric_attributes[outer])) 
    #print(intersecting_attr)
    #create projection list
    projection_list = " , ".join(map(lambda attr : f"o.`{attr}` as `o.{attr}` , i.`{attr}` as `i.{attr}`", intersecting_attr))
    sqlDF = spark.sql("SELECT "+projection_list+" FROM " +outer +" o JOIN "+ \
                            inner+ " i " + join_condition+")")
    # filter out null tupels with null values
    sqlDF = sqlDF.dropna(subset=list(map(lambda cur_col : f"`{cur_col}`", sqlDF.columns)))
    # calculates null values in the table
    #sqlDF.select([count(when(isnan(f"`{c}`") | col(f"`{c}`").isNull(), c)).alias(f"`{c}`") for c in sqlDF.columns]).show()
    
    # calcultes basic statisitc for the attributes
    print_df_to_html(sqlDF.describe())

    attr_variations = pair_permutations_ordered(intersecting_attr)
    #print(attr_variations)

    # selsect specific attr_variation with a specific attribute included
    sel_attr = 'H'
    sel_attr_variations = list(filter(lambda x: x[0]==sel_attr, attr_variations))
    print(sel_attr_variations)


    #sel_attr_variations = [['H', "H"]]

    for curr_item in sel_attr_variations :
        first_attr = curr_item[0]
        second_attr = curr_item[1]    
        ws_calc = wasserstein_distance( \
               asarray(sqlDF.select(col(f"`o.{first_attr}`")).rdd.flatMap(lambda x: x).collect()),                  asarray(sqlDF.select(col(f"`i.{second_attr}`")).rdd.flatMap(lambda x: x).collect()))
        first_second_projection = sqlDF.select(col(f"`o.{first_attr}`"),col(f"`i.{second_attr}`"))
        cos_calc = 0.0
        cos_sim = RowMatrix(first_second_projection.rdd.map(list)).columnSimilarities()
        if cos_sim.entries.count()> 0: 
            cos_calc = cos_sim.entries.first().value
        else:
            # print(f"Cos calc not possible {outer} {first_attr} {first_second_projection.dtypes[0]} {inner} {second_attr} {first_second_projection.dtypes[1]}")
            cos_calc = float("NaN")
        result_dist_calc.append([outer, first_attr, inner, second_attr, \
                                 float(ws_calc), float(cos_calc) ])
result_dist_calcdf = spark.createDataFrame(result_dist_calc).toDF("OUTER","OUTER_ATTR","INNER","INNER_ATTR","EMD", "COS")
result_dist_calcdf.coalesce(1).write.format("csv").mode("overwrite")\
                    .option("header","true").save("results/emd_cos_result_dist_calcs")
result_dist_calcdf.show()

[['H', 'SLG'], ['H', 'pwRC.'], ['H', 'BIP'], ['H', 'BABIP'], ['H', 'GIDP'], ['H', 'X1B'], ['H', 'LD'], ['H', 'wOBA'], ['H', 'wRAA'], ['H', 'HR'], ['H', 'AB'], ['H', 'wRC'], ['H', 'PA'], ['H', 'FB'], ['H', 'TB'], ['H', 'X3B'], ['H', 'wRC.'], ['H', 'year'], ['H', 'LD.'], ['H', 'OBP'], ['H', 'PU'], ['H', 'X2B'], ['H', 'PU.'], ['H', 'AVG'], ['H', 'GB.'], ['H', 'HBP'], ['H', 'SF'], ['H', 'SH'], ['H', 'FB.'], ['H', 'GB'], ['H', 'Number of Records'], ['H', 'ISO'], ['H', 'H']]
[['H', 'SLG'], ['H', 'pwRC.'], ['H', 'BIP'], ['H', 'BABIP'], ['H', 'GIDP'], ['H', 'X1B'], ['H', 'LD'], ['H', 'wOBA'], ['H', 'wRAA'], ['H', 'HR'], ['H', 'AB'], ['H', 'wRC'], ['H', 'PA'], ['H', 'FB'], ['H', 'TB'], ['H', 'X3B'], ['H', 'wRC.'], ['H', 'year'], ['H', 'LD.'], ['H', 'OBP'], ['H', 'PU'], ['H', 'X2B'], ['H', 'PU.'], ['H', 'AVG'], ['H', 'GB.'], ['H', 'HBP'], ['H', 'SF'], ['H', 'SH'], ['H', 'FB.'], ['H', 'GB'], ['H', 'Number of Records'], ['H', 'ISO'], ['H', 'H']]
[['H', 'SLG'], ['H', 'pwRC.'], ['H', 'BB.'], ['H', '

In [56]:
spark.stop()